### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 7, 17), '2022-29')

### Process after last business day, today must be last business day

In [2]:
today = today - timedelta(days=2)
wkago = today - timedelta(days=7)
today, wkago

(datetime.date(2022, 7, 15), datetime.date(2022, 7, 8))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,GLOBAL,2022-06-27,18.40


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'GLOBAL'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,193,GLOBAL,SET50 / SETCLMV / SETTHSI / SETWB,17.20,23.70,16.80,23.37,3.99,"4,801.74","82,589.87",279.44,0.83,193,2017-07-23 07:24:46.860402,2022-07-16 12:09:53.626998


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "GLOBAL" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,GLOBAL,2022-07-15,17.20,17.50,16.90,"11,124,366",17.00
1,GLOBAL,2022-07-14,16.90,17.10,16.80,"8,493,182",16.90
2,GLOBAL,2022-07-12,16.90,17.20,16.90,"12,002,208",17.20
3,GLOBAL,2022-07-11,17.30,17.60,17.00,"13,744,352",17.50
4,GLOBAL,2022-07-08,17.60,18.00,17.40,"12,521,258",17.90


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-07-15' AND name IN ('ASP', 'BCH', 'DCC', 'DIF', 'DOHOME', 'GLOBAL', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TPIPL', 'WHAIR', 'WHART')
ORDER BY name, date


26

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-07-08' AND name IN ('ASP', 'BCH', 'DCC', 'DIF', 'DOHOME', 'GLOBAL', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TPIPL', 'WHAIR', 'WHART') 
ORDER BY name


26

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(26, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASP,3.04,3.14,-3.18
1,BCH,20.40,20.20,0.99
2,DCC,2.76,2.76,0.00
3,DIF,13.20,13.30,-0.75
4,DOHOME,14.80,14.90,-0.67
5,GLOBAL,17.20,17.60,-2.27
6,JASIF,8.80,8.80,0.00
7,KCE,54.00,55.00,-1.82
8,MAKRO,34.75,35.50,-2.11
9,MCS,11.00,11.30,-2.65


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
13,RCL,32.75,37.25,-12.08%,Worse
12,PTTGC,43.50,45.50,-4.40%,Worse
22,TOP,49.25,51.25,-3.90%,Worse
0,ASP,3.04,3.14,-3.18%,Worse
11,ORI,9.65,9.95,-3.02%,Worse
9,MCS,11.00,11.30,-2.65%,Worse
5,GLOBAL,17.20,17.60,-2.27%,Worse
8,MAKRO,34.75,35.50,-2.11%,Worse
14,SCC,371.00,379.00,-2.11%,Worse
25,WHART,10.00,10.20,-1.96%,Worse


In [16]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,65.38%
Better,19.23%
No Change,15.38%


In [17]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(250, 8)

In [18]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [19]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(26, 13)

In [20]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
12,PTTGC,43.50,45.50,-4.40%,Worse,SET50
22,TOP,49.25,51.25,-3.90%,Worse,SET50
5,GLOBAL,17.20,17.60,-2.27%,Worse,SET50
14,SCC,371.00,379.00,-2.11%,Worse,SET50
7,KCE,54.00,55.00,-1.82%,Worse,SET50
20,TISCO,86.75,86.50,0.29%,Better,SET50


In [21]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,83.33%
Better,16.67%


In [22]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
13,RCL,32.75,37.25,-12.08%,Worse,SET100
11,ORI,9.65,9.95,-3.02%,Worse,SET100
4,DOHOME,14.80,14.90,-0.67%,Worse,SET100
1,BCH,20.40,20.20,0.99%,Better,SET100
18,STA,20.30,20.00,1.50%,Better,SET100
19,SYNEX,17.40,16.70,4.19%,Better,SET100


In [23]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,50.00%
Worse,50.00%


In [24]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
0,ASP,3.04,3.14,-3.18%,Worse,SET999
9,MCS,11.00,11.30,-2.65%,Worse,SET999
8,MAKRO,34.75,35.50,-2.11%,Worse,SET999
25,WHART,10.00,10.20,-1.96%,Worse,SET999
21,TMT,8.05,8.20,-1.83%,Worse,SET999
24,WHAIR,7.25,7.35,-1.36%,Worse,SET999
16,SENA,3.86,3.90,-1.03%,Worse,SET999
15,SCCC,148.00,149.50,-1.00%,Worse,SET999
3,DIF,13.20,13.30,-0.75%,Worse,SET999
2,DCC,2.76,2.76,0.00%,No Change,SET999


In [25]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,64.29%
No Change,28.57%
Better,7.14%


In [26]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt


In [27]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
